In [1]:
"""
Camila modification --

DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut
A Mathis, alexander.mathis@bethgelab.org
M Mathis, mackenzie@post.harvard.edu
This script evaluates a trained model at a particular state on the data set (images)
and stores the results in a pandas dataframe.
Script called from Step1_EvaluateModelonDataset.py
"""

import sys
import os
subfolder = os.getcwd().split('Evaluation-Tools')[0]
sys.path.append(subfolder)


# add parent directory: (where nnet & config are!)
sys.path.append(subfolder + "pose-tensorflow")
sys.path.append(subfolder + "Generating_a_Training_Set")

from myconfig import Task, date, Shuffles, scorer, TrainingFraction,snapshotindex

# Deep-cut dependencies
from config import load_config
from nnet import predict
from dataset.pose_dataset import data_to_input

# Dependencies for anaysis
import pickle
import skimage
import numpy as np
import pandas as pd
from skimage import io
import skimage.color
import auxiliaryfunctions
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2


print("Starting evaluation") #, sys.argv)


Starting evaluation


## To Define

In [2]:
# tanyaMay24-trainset70shuffle2  -  DeepCut_resnetresnet_50_70shuffle2_130000forTask:tanya
#snapshot-130000

snapshot =  'snapshot-130000'

shuffleIndex =  2

trainFractionIndex =  0  #int(sys.argv[3])

test_images_path = '/is/ps2/calvarez2/DeepLabCut/test_images/images'
labeled_images_path = '/is/ps2/calvarez2/DeepLabCut/test_images/labeled_images'

max_input_size = 1000.0

## -------------------

In [3]:
shuffle=Shuffles[shuffleIndex]
trainFraction=TrainingFraction[trainFractionIndex]

basefolder = os.path.join('..','pose-tensorflow','models')
folder = os.path.join('UnaugmentedDataSet_' + Task + date)

In [4]:
#######################################################################
# Load and setup CNN part detector as well as its configuration
#######################################################################

experimentname = Task + date + '-trainset' + str(int(trainFraction * 100)) + 'shuffle' + str(shuffle)
cfg = load_config(os.path.join(basefolder , experimentname , 'test' ,"pose_cfg.yaml"))
modelfolder = os.path.join(basefolder, experimentname)

cfg['init_weights'] = os.path.join(modelfolder,'train',snapshot)

trainingsiterations = (
    cfg['init_weights'].split('/')[-1]).split('-')[-1]
DLCscorer = 'DeepCut' + "_resnet" + str(cfg["net_type"]) + "_" + str(
    int(trainFraction *
        100)) + 'shuffle' + str(shuffle) + '_' + str(trainingsiterations) + "forTask:" + Task

print("Running ", DLCscorer, " with # of trainingiterations:", trainingsiterations)


('Running ', 'DeepCut_resnetresnet_50_70shuffle2_130000forTask:tanya', ' with # of trainingiterations:', '130000')


In [5]:
try:
    Data = pd.read_hdf(os.path.join("Results",DLCscorer + '.h5'),'df_with_missing')
    print("This net has already been evaluated!")
except :
    # Specifying state of model (snapshot / training state)
    cfg['init_weights'] = os.path.join(modelfolder,'train',snapshot)
    
    sess, inputs, outputs = predict.setup_pose_prediction(cfg)
    
    
    images_names = os.listdir(test_images_path)
    
    Numimages = len(images_names) #len(Data.index)
    PredicteData = np.zeros((Numimages,3 * len(cfg['all_joints_names'])))
    Testset = np.zeros(Numimages)
    
    print("Analyzing data...")
    
    ##################################################
    # Compute predictions over images
    ##################################################
    
    for imageindex, imagename in tqdm(enumerate(images_names)):
        image = io.imread(os.path.join(test_images_path, imagename),mode='RGB')
        image = skimage.color.gray2rgb(image)
        height, width, channels = image.shape
        if(width * height > max_input_size*max_input_size):
                factor = (max_input_size*0.9)/max(height, width)
                image = cv2.resize(image, None, fx = factor, fy = factor, interpolation = cv2.INTER_CUBIC)
        
        image_batch = data_to_input(image)
        
        
        # Compute prediction with the CNN
        outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
        scmap, locref = predict.extract_cnn_output(outputs_np, cfg)
        
        # Extract maximum scoring location from the heatmap, assume 1 person
        pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
        PredicteData[imageindex, :] = pose.flatten(
        )  # NOTE: thereby     cfg_test['all_joints_names'] should be same order as bodyparts!
      
        index = pd.MultiIndex.from_product(
            [[DLCscorer], cfg['all_joints_names'], ['x', 'y', 'likelihood']],
            names=['scorer', 'bodyparts', 'coords'])

        # Saving results:
        auxiliaryfunctions.attempttomakefolder("Results")

        DataMachine = pd.DataFrame(
            PredicteData, columns=index, index= images_names )
        DataMachine.to_hdf(os.path.join("Results",'Test_sample_'+DLCscorer + '.h5'),'df_with_missing',format='table',mode='w')
        print("Done and results stored for snapshot: ", snapshot)        

        

Instructions for updating:
Pass is_training directly to the network instead of the arg_scope.


Instructions for updating:
Pass is_training directly to the network instead of the arg_scope.


INFO:tensorflow:Restoring parameters from ../pose-tensorflow/models/tanyaMay24-trainset70shuffle2/train/snapshot-130000


INFO:tensorflow:Restoring parameters from ../pose-tensorflow/models/tanyaMay24-trainset70shuffle2/train/snapshot-130000
0it [00:00, ?it/s]

Analyzing data...


...

Folder already exists!
('Done and results stored for snapshot: ', 'snapshot-130000')


3it [00:01,  1.89it/s]

Folder already exists!
('Done and results stored for snapshot: ', 'snapshot-130000')
Folder already exists!
('Done and results stored for snapshot: ', 'snapshot-130000')


4it [00:02,  1.96it/s]

Folder already exists!
('Done and results stored for snapshot: ', 'snapshot-130000')


In [6]:
DataMachine

scorer          DeepCut_resnetresnet_50_70shuffle2_130000forTask:tanya  \
bodyparts                                                     tail tip   
coords                                                               x   
download.jpeg                                           245.382316       
frame00.png                                             409.653399       
frame016.png                                            345.447287       
download_1.jpeg                                         182.991820       

scorer                                                                  \
bodyparts                              back right shoulder               
coords                    y likelihood                   x           y   
download.jpeg    100.015517   0.265371          185.128766   74.701450   
frame00.png      344.335851   0.986123          443.690574  329.763581   
frame016.png     437.938692   0.801736          456.827516  376.762300   
download_1.jpeg  129.773887   0.004649          581.161192  368.587721   

scorer                                                                         \
bodyparts                   tail start                        front left knee   
coords          likelihood           x           y likelihood               x   
download.jpeg     0.395937  223.865395   50.322142   0.096255      121.818414   
frame00.png       0.985115  419.713856  300.987465   0.985115      487.435950   
frame016.png      0.351176  389.137235  273.189515   0.999952      711.372278   
download_1.jpeg   0.133978  489.237836  203.015867   0.047165      314.874008   

scorer             ...                                                 \
bodyparts          ...     back right foot back left knee               
coords             ...          likelihood              x           y   
download.jpeg      ...            0.118286     231.784291  113.234206   
frame00.png        ...            0.993838     436.571662  359.365816   
frame016.png       ...            0.243947     426.258573  455.421776   
download_1.jpeg    ...            0.021963     308.864637  510.295025   

scorer                                                                         \
bodyparts                    right ear                        back left ankle   
coords          likelihood           x           y likelihood               x   
download.jpeg     0.014248   61.343495   11.453465   0.084508      175.521053   
frame00.png       0.999783  495.330883  272.651004   0.999657      436.012750   
frame016.png      0.521080  794.394336  122.110319   0.919876      346.732061   
download_1.jpeg   0.012532  193.412937   34.499790   0.200286      591.278532   

scorer                                  
bodyparts                               
coords                    y likelihood  
download.jpeg    153.814428   0.027130  
frame00.png      375.224007   0.999488  
frame016.png     445.309906   0.012455  
download_1.jpeg  590.505276   0.002738  

[4 rows x 75 columns]

In [7]:
DLCscorer

'DeepCut_resnetresnet_50_70shuffle2_130000forTask:tanya'

# --- one image ----


In [8]:
import os.path
import sys
#sys.path.append(os.getcwd().split('Generating_a_Training_Set')[0])


from skimage import io
import matplotlib
%matplotlib inline
# matplotlib.use('Agg')
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [5,3]
from myconfig import Task, filename, bodyparts, Scorers
from myconfig import scorer as cfg_scorer

Labels = ['.', '+', '*']  # order of labels for different scorers

# https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
Colorscheme = get_cmap(len(bodyparts))

comparisonbodyparts = list(set(DataMachine.columns.get_level_values(1)))

scale = 1  # for plotting
msize = 15   #size of labels

scorer_machine = DLCscorer

## To Define: img name

In [9]:
# imagename = 'frame016.png' #'download_1.jpeg.jpeg' #'frame016.png'  #'frame016.png'

for imageindex, imagename in tqdm(enumerate(images_names)):


    imindex = np.where(
                np.array(DataMachine.index.values) == imagename)[0]

    # image = io.imread('/is/ps2/calvarez2/DeepLabCut/pose-tensorflow/models/UnaugmentedDataSet_tanyaMay24/data-tanya/' + imagename)
    image = io.imread(os.path.join(test_images_path, imagename),mode='RGB')
    image = skimage.color.gray2rgb(image) 
    height, width, channels = image.shape
    if(width * height > max_input_size*max_input_size):
        factor = (max_input_size*0.9)/max(height, width)
        image = cv2.resize(image, None, fx = factor, fy = factor, interpolation = cv2.INTER_CUBIC)
    
    plane_name , extenstion = imagename.split('.')

    if extenstion != 'png':
        imagename = plane_name + '.png'


    plt.axis('off')

    if np.ndim(image)==2:
        h, w = np.shape(image)
    else:
        h, w, nc = np.shape(image)

    plt.figure(
        frameon=False, figsize=(w * 1. / 100 * scale, h * 1. / 100 * scale))
    plt.subplots_adjust(
        left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)

    plt.imshow(image, 'bone')
    for cc, scorer in enumerate(Scorers):
        for c, bp in enumerate(comparisonbodyparts):

            plt.plot(   
                    DataMachine[scorer_machine][bp]['x'].values[imindex],
                    DataMachine[scorer_machine][bp]['y'].values[imindex],
                    Labels[0],
                    color=Colorscheme(c),#color=Colorscheme[c],#color=Colorscheme[0],
                    alpha=.5,
                    ms=msize)

    plt.xlim(0, w)
    plt.ylim(0, h)
    plt.axis('off')
    plt.subplots_adjust(
        left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    plt.gca().invert_yaxis()
    plt.savefig(labeled_images_path + '/' + imagename , dpi=100) #dpi=100 = original size
    plt.close("all")



4it [00:02,  1.99it/s]
